# Inference YOLOv8

In [44]:
# from PIL import Image, ImageDraw, ImageFont
# import argparse
# from pathlib import Path
# from ultralytics import YOLO

# # Function to plot bounding boxes
# def plot_boxes(draw, boxes, class_labels, color):
#     for box in boxes:
#         cls, xyxy = box
#         x1, y1, x2, y2 = xyxy

#         # Draw bounding box rectangle
#         draw.rectangle([x1, y1, x2, y2], outline=color, width=3)

#         # Add class text
#         font = ImageFont.load_default()
#         text = class_labels[cls]
#         text_bbox = draw.textbbox((0, 0), text, font=font)
#         text_width = text_bbox[2] - text_bbox[0]
#         text_height = text_bbox[3] - text_bbox[1]
#         text_x = x1
#         text_y = y1 - text_height

#         if text_y < 0:
#             text_y = 0

#         # Draw filled rectangle for text background
#         draw.rectangle([text_x, text_y, text_x + text_width, text_y + text_height], fill=color)

#         # Draw text on top of the rectangle
#         draw.text((text_x, text_y), text, fill='white', font=font)

# def display_ground_truths_and_predictions(optimizer, image_path, label_path, class_labels):
#     # Determine model directory based on optimizer
#     if optimizer.lower() == 'adam':
#         model_path = 'D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb/yolov8n-obb - nano adam/weights/best.pt'  # Replace with your actual path
#     elif optimizer.lower() == 'sgd':
#         model_path = 'D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb/yolov8n-obb - nano sgd/weights/best.pt'  # Replace with your actual path
#     else:
#         raise ValueError(f"Optimizer '{optimizer}' is not supported. Choose either 'adam' or 'sgd'.")

#     # Load YOLO model
#     model = YOLO(model_path)

#     # Load image
#     image = Image.open(image_path)
#     width, height = image.size
#     draw = ImageDraw.Draw(image)

#     # Load ground truths
#     label_file = label_path  # Assuming label_path directly points to the label file
#     if Path(label_file).exists():
#         with open(label_file, 'r') as f:
#             lines = f.readlines()
#             for line in lines:
#                 entries = line.split()
#                 cls = int(entries[0])
#                 coordinates = [(float(entries[i]) * width, float(entries[i+1]) * height) for i in range(1, len(entries), 2)]
#                 coordinates.append(coordinates[0])  # Close the polygon
#                 plot_boxes(draw, [(cls, coordinates)], class_labels, color='red')

#     # Run inference
#     results = model.predict(image)
    
#     # Plot predictions
#     for result in results:
#         print(result.obb)
#         # boxes = result[:, :4].tolist()  # Extract xyxy coordinates and convert to list
#         # scores = result[:, 4].tolist()   # Extract confidence scores
#         # classes = result[:, 5].tolist()  # Extract class indices
#         # for box, score, cls in zip(boxes, scores, classes):
#         #     plot_boxes(draw, [(cls, box)], class_labels, color='green')

#     # Display the image
#     image.show()

# if __name__ == "__main__":
#     optimizer = 'sgd'  # Replace with 'adam' or 'sgd'
#     image_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets/images/val/13143_lr-0.png"
#     label_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets/labels/val/13143_lr-0.txt"

#     # Class labels for display
#     class_labels = ['I arrhythmia', 'II arrhythmia', 'III arrhythmia', 'aVF arrhythmia', 'aVL arrhythmia', 'aVR arrhythmia',
#                     'V1 arrhythmia', 'V2 arrhythmia', 'V3 arrhythmia', 'V4 arrhythmia', 'V5 arrhythmia', 'V6 arrhythmia']

#     # Call function to display ground truths and predictions
#     display_ground_truths_and_predictions(optimizer, image_path, label_path, class_labels)


In [50]:
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO

# image_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets/images/val/13143_lr-0.png"
# label_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets/labels/val/13143_lr-0.txt"
image_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets_binarize/val/images/13143_lr-0_1.png"
label_path = "D:/Skripsweet/BISMILLAH_KELAR/datasets_binarize/val/labels/13143_lr-0_1.txt"

# Define class labels
labels = [
    'I arrhythmia', 'II arrhythmia', 'III arrhythmia', 
    'aVF arrhythmia', 'aVL arrhythmia', 'aVR arrhythmia',
    'V1 arrhythmia', 'V2 arrhythmia', 'V3 arrhythmia', 
    'V4 arrhythmia', 'V5 arrhythmia', 'V6 arrhythmia'
]

yolo_entries = []
with open(label_path, 'r') as f:
    yolo_entries = f.readlines()

boxes = []
for yolo_entry in yolo_entries:
    data = yolo_entry.split()
    label = int(data[0])
    box = list(map(float, data[1:]))
    boxes.append((label, box))  # Store label and box coordinates as tuple

# Function to plot bounding boxes and labels on an image
def plot_boxes(image_path, boxes, labels, color='red'):
    # Open the image using PIL
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    width, height = image.size

    for label, box in boxes:
        if len(box) == 8:
            x1, y1, x2, y2, x3, y3, x4, y4 = box  # Extract coordinates
            # Convert YOLO format to PIL format (x1, y1, x2, y2, x3, y3, x4, y4)
            xy = [(x1 * width, y1 * height), (x2 * width, y2 * height),
                  (x3 * width, y3 * height), (x4 * width, y4 * height), (x1 * width, y1 * height)]

            draw.line(xy, fill=color, width=3)
            
            # Draw filled background rectangle
            label_text = labels[label]
            font_size = 30
            font = ImageFont.truetype("arial.ttf", font_size)  # Adjust font type and size as needed
            
            # Calculate text size and position
            text_width, text_height = 200, 30
            # draw.textsize(label_text, font=font)
            text_x = x1 * width + 5
            text_y = y1 * height - text_height - 5  # Adjust for text position above the bounding box
            bg_color = 'white'  # Background color
            draw.rectangle([(text_x, text_y), 
                            (text_x + text_width, text_y + text_height)], fill=bg_color)
            
            # Draw label text with larger font size
            draw.text((text_x, text_y), label_text, fill=color, font=font)

    # Save or display the image with bounding boxes and labels
    image.show()  # Show the image with bounding boxes and labels

plot_boxes(image_path, boxes, labels)

In [2]:
from ultralytics import YOLO

In [17]:
image_path_color = "D:/Skripsweet/BISMILLAH_KELAR/datasets/images/val/13143_lr-0.png"
image_path_binarize = "D:/Skripsweet/BISMILLAH_KELAR/datasets_binarize/val/images/13143_lr-0_1.png"

In [18]:
# Load a pretrained YOLOv8n model
model = YOLO("D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb/yolov8n-obb - nano adam/weights/best.pt")

results = model.predict(image_path_color, save=True)
for result in results:
    boxes = result.boxes


image 1/1 D:\Skripsweet\BISMILLAH_KELAR\datasets\images\val\13143_lr-0.png: 512x640 17.1ms
Speed: 4.5ms preprocess, 17.1ms inference, 5.9ms postprocess per image at shape (1, 3, 512, 640)
Results saved to C:\Users\miran\runs\obb\predict21


In [19]:
model_sgd = YOLO("D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb/yolov8n-obb - nano sgd/weights/best.pt")
dir_sgd = 'D:/Skripsweet/BISMILLAH_KELAR/inference_result/color_sgd'

results_sgd = model_sgd.predict(image_path_color, save=True, save_dir=dir_sgd)
for result in results_sgd:
    boxes = result.boxes


image 1/1 D:\Skripsweet\BISMILLAH_KELAR\datasets\images\val\13143_lr-0.png: 512x640 21.8ms
Speed: 3.5ms preprocess, 21.8ms inference, 5.6ms postprocess per image at shape (1, 3, 512, 640)
Results saved to C:\Users\miran\runs\obb\predict22


In [20]:
model_adam_binary = YOLO("D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb_Adam/yolov8n-obb/weights/best.pt")
dir_adam_binary = 'D:/Skripsweet/BISMILLAH_KELAR/inference_result/binary_adam'

# Run inference on 'bus.jpg' and 'zidane.jpg'
results_adam_binary = model_adam_binary.predict(image_path_binarize, save=True, save_dir=dir_adam_binary)
for result in results_adam_binary:
    boxes = result.boxes


image 1/1 D:\Skripsweet\BISMILLAH_KELAR\datasets_binarize\val\images\13143_lr-0_1.png: 512x640 14.0ms
Speed: 4.0ms preprocess, 14.0ms inference, 6.0ms postprocess per image at shape (1, 3, 512, 640)
Results saved to C:\Users\miran\runs\obb\predict23


In [21]:
model_sgd_binary = YOLO("D:/Skripsweet/BISMILLAH_KELAR/runs/yolov8n-obb_SGD/yolov8n-obb/weights/best.pt")
dir_sgd_binary = 'D:/Skripsweet/BISMILLAH_KELAR/inference_result/binary_sgd'

# Run inference on 'bus.jpg' and 'zidane.jpg'
results_sgd_binary = model_sgd_binary.predict(image_path_binarize, save=True, save_dir=dir_sgd_binary)
for result in results_sgd_binary:
    boxes = result.boxes


image 1/1 D:\Skripsweet\BISMILLAH_KELAR\datasets_binarize\val\images\13143_lr-0_1.png: 512x640 20.5ms
Speed: 7.0ms preprocess, 20.5ms inference, 7.1ms postprocess per image at shape (1, 3, 512, 640)
Results saved to C:\Users\miran\runs\obb\predict24
